In [39]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [4]:
labels = np.load('processed_labels.npy')
data = np.load('processed_data.npy')

In [51]:
# Simple example CNN
# Bog standard image recognition toy architecture

model = keras.models.Sequential([
    keras.layers.Reshape([50, 50, 1], input_shape=[50, 50]),
    keras.layers.Conv2D(16, kernel_size=3, activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(32, kernel_size=3, activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(64),
    keras.layers.Dense(128),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 50, 50, 1)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 48, 48, 16)        160       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 22, 22, 32)        4640      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 11, 11, 32)       

In [48]:
# Convert data into a format we can actually feed into keras
# For this example, treat each image idependently of each other
# So we turn a [4000,200,50,50] array into [sample_size,50,50]
# And also generate the corresponding y array of [sample_size, output_size]

x_all = list()
y_all = list()
for i,x in enumerate(data[:200]):
    label = labels[i].tolist()
    if label == [0.5, 0.5]:
        continue
    x = x[30:175]
    x_all += x.tolist()
    y_all += [labels[i]]*145
    
x_all = np.asarray(x_all)
y_all = np.asarray(y_all)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

In [52]:
# Train the model

# Reset the state in case we run it more than one time
model.reset_states()

# Early stopping callback for when we don't progress any more
cb = keras.callbacks.EarlyStopping(
    monitor='accuracy', patience=4, restore_best_weights=True, min_delta=0.005)

imhistory = model.fit(x=x_train, y=y_train, epochs=75, validation_split=0.2, callbacks=[cb])

Epoch 1/50
566/566 [==============================] - 10s 15ms/step - loss: 0.5760 - accuracy: 0.7976 - val_loss: 0.2136 - val_accuracy: 0.8915
Epoch 2/50
566/566 [==============================] - 8s 15ms/step - loss: 0.2338 - accuracy: 0.8541 - val_loss: 0.1734 - val_accuracy: 0.8798
Epoch 3/50
566/566 [==============================] - 8s 15ms/step - loss: 0.2020 - accuracy: 0.8668 - val_loss: 0.1592 - val_accuracy: 0.8782
Epoch 4/50
566/566 [==============================] - 8s 15ms/step - loss: 0.1816 - accuracy: 0.8720 - val_loss: 0.1502 - val_accuracy: 0.8924
Epoch 5/50
566/566 [==============================] - 8s 15ms/step - loss: 0.1673 - accuracy: 0.8727 - val_loss: 0.1333 - val_accuracy: 0.8888
Epoch 6/50
566/566 [==============================] - 8s 15ms/step - loss: 0.1574 - accuracy: 0.8778 - val_loss: 0.1215 - val_accuracy: 0.9109
Epoch 7/50
566/566 [==============================] - 8s 15ms/step - loss: 0.1535 - accuracy: 0.8821 - val_loss: 0.1156 - val_accuracy: 0.897

In [53]:
model.evaluate(x_test, y_test)

177/177 [==============================] - 1s 7ms/step - loss: 0.0599 - accuracy: 0.9045


[0.05989319831132889, 0.9045093059539795]

In [54]:
model.save('models/simple_cnn_example.hdf5')